<a href="https://colab.research.google.com/github/Raghnus/Kaggle_Comp/blob/main/Titanic/KaggleTitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import  XGBClassifier

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PTCienciaDatos/PandasParaManipulacionDeDatos/Kaggle/Titanic/Titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df.groupby(['Embarked'])['PassengerId'].count()

Embarked
C    168
Q     77
S    644
Name: PassengerId, dtype: int64

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
df_train = df[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived',]]

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          714 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Embarked     889 non-null    object 
 8   Survived     891 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [ ]:
(df_train['Age'].isna().sum()*100)/len(df)

19.865319865319865

In [ ]:
df_train = df_train.dropna()

In [ ]:
le = LabelEncoder()

le.fit_transform(df_train.Sex.unique())
df_train.Sex =le.transform(df_train.Sex)
le.fit_transform(df_train.Embarked.unique())
df_train.Embarked = le.transform(df_train.Embarked)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1,3,1,22.0,1,0,7.2500,2,0
1,2,1,0,38.0,1,0,71.2833,0,1
2,3,3,0,26.0,0,0,7.9250,2,1
3,4,1,0,35.0,1,0,53.1000,2,1
4,5,3,1,35.0,0,0,8.0500,2,0
...,...,...,...,...,...,...,...,...,...
885,886,3,0,39.0,0,5,29.1250,1,0
886,887,2,1,27.0,0,0,13.0000,2,0
887,888,1,0,19.0,0,0,30.0000,2,1
889,890,1,1,26.0,0,0,30.0000,0,1


In [ ]:
df_train.corr()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
PassengerId,1.000000,-0.035609,0.024674,0.033681,-0.082704,-0.011672,0.009655,-0.001836,0.029526
Pclass,-0.035609,1.000000,0.150826,-0.365902,0.065187,0.023666,-0.552893,0.244145,-0.356462
Sex,0.024674,0.150826,1.000000,0.099037,-0.106296,-0.249543,-0.182457,0.109639,-0.536762
Age,0.033681,-0.365902,0.099037,1.000000,-0.307351,-0.187896,0.093143,-0.032565,-0.082446
SibSp,-0.082704,0.065187,-0.106296,-0.307351,1.000000,0.383338,0.139860,0.033064,-0.015523
Parch,-0.011672,0.023666,-0.249543,-0.187896,0.383338,1.000000,0.206624,0.011803,0.095265
Fare,0.009655,-0.552893,-0.182457,0.093143,0.139860,0.206624,1.000000,-0.283510,0.266100
Embarked,-0.001836,0.244145,0.109639,-0.032565,0.033064,0.011803,-0.283510,1.000000,-0.181979
Survived,0.029526,-0.356462,-0.536762,-0.082446,-0.015523,0.095265,0.266100,-0.181979,1.000000


In [ ]:
X = df_train.loc[:, (df_train.columns!='Survived')]
y = df_train['Survived'] 

In [ ]:
X.shape

(712, 8)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=3)

In [ ]:
pipe = make_pipeline(StandardScaler(),XGBClassifier())
pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier', XGBClassifier())])

In [ ]:
print('Train score ',pipe.score(X_train,y_train))
print('Test score',pipe.score(X_test,y_test))

Train score  0.9044943820224719
Test score 0.8089887640449438


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/PTCienciaDatos/PandasParaManipulacionDeDatos/Kaggle/Titanic/Titanic/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
df_test = df_test[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [ ]:
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          332 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         417 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


In [ ]:
df_test.Fare = df_test.Fare.interpolate()
df_test.Age = df_test.Age.interpolate()

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


In [ ]:
le.fit_transform(df_test.Sex.unique())
df_test.Sex =le.transform(df_test.Sex)
le.fit_transform(df_test.Embarked.unique())
df_test.Embarked = le.transform(df_test.Embarked)

In [ ]:
predicciones=pipe.predict(df_test)

In [ ]:
resultados = pd.DataFrame({'PassengerId':df_test['PassengerId'], 'Survived':predicciones})
resultados

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
resultados.to_csv('/content/drive/MyDrive/PTCienciaDatos/PandasParaManipulacionDeDatos/Kaggle/Titanic/resultados.csv', index=False)